<a href="https://colab.research.google.com/github/Siddhartha96123/GCHS/blob/main/GCHS_1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

WELCOME to the **General** Executable Conformational Heterogeneity Scanner.

It is capable of performing a '***PDB vs PDB***' conformational heterogeneity scanning of ***same enzyme from the SAME or two different organisms***.

**IT IS NOT MEANT FOR ALIGNING TWO DIFFERENT ENZYME CLASSES WITH EACH OTHER.** *Please refrain from doing so.*

In [ ]:
# Required libraries installation
!pip install biopython

Now that the dependencies are done : Lets move onto the actual script.

The script will at the tail end of this code - ask for the **Reference PDB** first - which shall be needed to be uploaded from your local system, followed by the **Target PDB**.

Green tick on the left side of the Play button implies this part has been executed completely without any errors. Herein, we are :

Defining the function for uploading files, Parsing, aligning.

Defining the function for aligning the structures.

Defining the function for calculating ***RMSD/Res***

Defining the function to map the values on a an excel sheet.

Defining the function for plotting the values from the excel to a Graph.

In [ ]:
from Bio.PDB import PDBParser, Superimposer, is_aa
from Bio import pairwise2
from Bio.SeqUtils import seq1
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files
import os
import time

# Function to upload files in Colab
def upload_files():
    """Uploads files in Colab and returns file paths."""
    uploaded = files.upload()
    file_paths = list(uploaded.keys())
    return file_paths

def extract_sequence_and_residues(structure):
    """Extracts the amino acid sequence, residues and their B-factors from the PDB structure."""
    sequence = ""
    residues = []
    b-factors = []

    for chain in structure[0]:
        for residue in chain:
            if is_aa(residue, standard=True):  # Check if residue is an amino acid
                sequence += seq1(residue.get_resname())  # Get one-letter code for the residue
                residues.append(residue)

                # Extract B-factor (taking average if multiple atoms per residue)
                atoms = [atom for atom in residue if atom.get_bfactor() is not None]
                avg_b_factor = np.mean([atom.get_bfactor() for atom in atoms]) if atoms else None
                b_factors.append(avg_b_factor)

    return sequence, residues, b_factors

def align_sequences(seq1, seq2):
    """Aligns two sequences using pairwise global alignment (globalxx) and returns the alignment and sequence identity."""
    alignments = pairwise2.align.globalxx(seq1, seq2)
    alignment = alignments[0]
    matches = sum(res1 == res2 for res1, res2 in zip(alignment[0], alignment[1]))
    sequence_identity = matches / min(len(seq1), len(seq2))
    print(f"Sequence Identity: {sequence_identity * 100:.2f}%")
    return alignment  # Return the best alignment

def map_residues_by_alignment(alignment, ref_residues, target_residues):
    """Maps residues based on sequence alignment."""
    ref_aligned_residues, target_aligned_residues = [], []
    ref_aligned_b_factors, target_aligned_b_factors = [], []

    ref_index, target_index = 0, 0

    for ref_aa, tgt_aa in zip(alignment[0], alignment[1]):
        if ref_aa != '-' and tgt_aa != '-':
            # Both residues are aligned, add their corresponding residues to the list
            ref_aligned_residues.append(ref_residues[ref_index])
            target_aligned_residues.append(target_residues[target_index])
            ref_aligned_b_factors.append(ref_b_factors[ref_index])
            target_aligned_b_factors.append(target_b_factors[target_index])

        if ref_aa != '-':
            ref_index += 1  # Move to the next residue in the reference sequence
        if tgt_aa != '-':
            target_index += 1  # Move to the next residue in the target sequence

    return ref_aligned_residues, target_aligned_residues, ref_aligned_b_factors, target_aligned_b_factors

def align_structures(reference_path, target_path):
    """Aligns two PDB structures based on their C-alpha atoms and extracts B-factor information."""
    parser = PDBParser(QUIET=True)

    # Load the reference structure
    print("Loading reference structure...")
    reference_structure = parser.get_structure("reference", reference_path)

    # Load the target structure
    print("Loading target structure...")
    target_structure = parser.get_structure("target", target_path)

    # Extract sequences and residues from both structures
    print("Extracting sequences, residues, and B-factors...")
    ref_seq, ref_residues, ref_b_factors = extract_sequence_and_residues(reference_structure)
    tgt_seq, target_residues, target_b_factors = extract_sequence_and_residues(target_structure)

    # Align sequences globally
    print("Aligning sequences...")
    alignment = align_sequences(ref_seq, tgt_seq)

    # Map residues based on alignment
    print("Mapping aligned residues...")
    ref_aligned_residues, target_aligned_residues, ref_aligned_b_factors, target_aligned_b_factors = map_residues_by_alignment(
        alignment, ref_residues, target_residues, ref_b_factors, target_b_factors)

    # Initialize the superimposer
    print("Superimposing structures...")
    superimposer = Superimposer()
    reference_atoms = [residue['CA'] for residue in ref_aligned_residues if 'CA' in residue]
    target_atoms = [residue['CA'] for residue in target_aligned_residues if 'CA' in residue]

    # Check if atoms are available for superimposition
    if not reference_atoms or not target_atoms:
        raise ValueError("No C-alpha atoms available for reference superimposition. Please check the structures.")

    # Perform the superimposition
    superimposer.set_atoms(reference_atoms, target_atoms)
    superimposer.apply(target_structure[0].get_atoms())

    return reference_structure, target_structure, ref_aligned_residues, target_aligned_residues, ref_aligned_b_factors, target_aligned_b_factors

def calculate_distances(ref_aligned_residues, target_aligned_residues):
    """Calculates distances between C-alpha atoms of aligned residues."""
    distances = []

    for ref_residue, target_residue in zip(ref_aligned_residues, target_aligned_residues):
        if 'CA' in ref_residue and 'CA' in target_residue:
            ref_atom = ref_residue['CA']
            target_atom = target_residue['CA']
            distance = np.linalg.norm(ref_atom.get_coord() - target_atom.get_coord())
            distances.append((ref_residue.get_id()[1], target_residue.get_id()[1], distance))

    return distances

def save_distances_to_excel(distances, ref_b_factors, target_b_factors, output_path="GCHS_PDB_RMSF.xlsx"):
    """Saves the calculated distances and B-factors to an Excel file."""
    df = pd.DataFrame({
        "Reference Residue": [d[0] for d in distances],
        "Aligned Residue": [d[1] for d in distances],
        "RMSF Distance": [d[2] for d in distances],
        "Reference B-factor": ref_b_factors,
        "Target B-factor": target_b_factors
    })

    df.to_excel(output_path, index=False)
    print(f"Results saved to {output_path}")

def plot_rmsf(distances, output_plot_path="GCHS_PDB_RMSF_plot.png", rmsf_threshold=None):
    """Plots the RMSF values and saves the plot."""
    reference_residues = [d[0] for d in distances]
    rmsf_values = [d[2] for d in distances]

    # Apply RMSF threshold if specified
    if rmsf_threshold:
        reference_residues = [ref for ref, rmsf in zip(reference_residues, rmsf_values) if rmsf <= rmsf_threshold]
        rmsf_values = [rmsf for rmsf in rmsf_values if rmsf <= rmsf_threshold]

    plt.scatter(reference_residues, rmsf_values)
    plt.xlabel("Reference Residue")
    plt.ylabel("RMSF (Å)")
    plt.title("Root Mean Square Fluctuation")
    plt.savefig(output_plot_path)
    print(f"RMSF plot saved to {output_plot_path}")
    plt.close()

def plot_b_factors(ref_b_factors, target_b_factors, output_plot_path="GCHS_BFactors_plot.png"):
    """Plots the B-factor trends of the reference and target structures."""
    plt.plot(ref_b_factors, label="Reference B-factors", linestyle='-', marker='o')
    plt.plot(target_b_factors, label="Target B-factors", linestyle='-', marker='s')

    plt.xlabel("Residue Index")
    plt.ylabel("B-factor")
    plt.title("B-factor Trends of Reference and Target Structures")
    plt.legend()
    plt.savefig(output_plot_path)
    print(f"B-factor trend plot saved to {output_plot_path}")
    plt.close()

The green tick on the above block here signals that the functions to perform every associated funciton within this tool have been Defined & Initialised
We can finally now, move to uploading the **SINGLE CHAIN PDBs** for all further analyses.

If you dont have your PDB split into its constituent PDBs - please go to https://colab.research.google.com/github/Siddhartha96123/GCHS/blob/main/GCHS_CHAIN_EXTRACT.ipynb

This aforementioned link is part of the repo for this GCHS tool on my GitHub page as well, if this causes any inconvinience.

In [ ]:
# Upload the PDB files individually
print("Please upload the reference PDB file.")
reference_pdb = upload_files()[0]

print("Please upload the target PDB file.")
target_pdb = upload_files()[0]

try:
    # Align structures
    reference_structure, aligned_structure, ref_aligned_residues, target_aligned_residues, ref_aligned_b_factors, target_aligned_b_factors = align_structures(reference_pdb, target_pdb)

    # Calculate distances between atoms
    distances = calculate_distances(ref_aligned_residues, target_aligned_residues)

    # Save results to Excel
    save_results_to_excel(distances, ref_aligned_b_factors, target_aligned_b_factors)

    # Plot RMSF
    plot_rmsf(distances)

    # Plot B-factor trends
    plot_b_factors(ref_aligned_b_factors, target_aligned_b_factors)

    # Download results
    print("Script execution completed. Downloading results...")
    files.download("GCHS_PDB_Analysis.xlsx")
    time.sleep(2)
    files.download("GCHS_BFactors_plot.png")

except Exception as e:
    print("An error occurred:", str(e))

Please write to me @ siddhartha96123@gmail.com if there are any concerns that you face.